In [6]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
library("stringr")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,731602,39.1,1330087,71.1,1330087,71.1
Vcells,1561685,12.0,1181718638,9015.8,1386353168,10577.1


In [7]:
kdatasetinput <- "c2_reparacEstad_fe_inicial.csv.gz"
kdatasetoutput <- "c2_reparacEstad_fe_inicialymiomed.csv.gz"

In [8]:
setwd("~/buckets/b1/")

#cargo el dataset
dataset  <- fread(paste0("./datasets/",kdatasetinput))   #donde entreno# FIN Parametros del script

In [9]:

AgregarVariables  <- function( dataset )
{
  gc()
 
############# FE 1 ####################
   
  #Aqui debe usted agregar sus propias nuevas variables           
    dataset[ , prop_vida_en_banco          := cliente_antiguedad/(12*cliente_edad)]
    
   
    #Prestamos
    dataset[ , cprestamos                 := rowSums( cbind( cprestamos_personales, cprestamos_prendarios, cprestamos_hipotecarios) , na.rm=TRUE ) ]
    dataset[ , mprestamos                 := rowSums( cbind( mprestamos_personales, mprestamos_prendarios, mprestamos_hipotecarios) , na.rm=TRUE ) ]
    
    #Plazos Fijos
    dataset[ , mplazo_fijo                := rowSums( cbind( mplazo_fijo_pesos, mplazo_fijo_dolares) , na.rm=TRUE ) ]

    #Inversiones
    dataset[ , minversion1                := rowSums( cbind( minversion1_pesos, minversion1_dolares) , na.rm=TRUE ) ]
    dataset[ , cinversion12                := rowSums( cbind( cinversion1, cinversion2) , na.rm=TRUE ) ]
    dataset[ , minversion12                := rowSums( cbind( minversion1, minversion2) , na.rm=TRUE ) ]
    dataset[ , cinversion_total                := rowSums( cbind( cinversion12, cplazo_fijo) , na.rm=TRUE ) ]
    dataset[ , minversion_total                := rowSums( cbind( minversion12, mplazo_fijo) , na.rm=TRUE ) ]

    #Seguros
    dataset[ , cseguro                := rowSums( cbind( cseguro_vida, cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales) , na.rm=TRUE ) ]

    #Debitos automaticos
    dataset[ , cdebitos_automaticos                := rowSums( cbind( ccuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos ) , na.rm=TRUE ) ]
    dataset[ , mdebitos_automaticos                := rowSums( cbind( mcuenta_debitos_automaticos,mttarjeta_visa_debitos_automaticos,mttarjeta_master_debitos_automaticos ) , na.rm=TRUE ) ]

    #Servicios
    dataset[ , cpago_servicios_total                := rowSums( cbind( cpagodeservicios,cpagomiscuentas ) , na.rm=TRUE ) ]
    dataset[ , mpago_servicios_total                := rowSums( cbind( mpagodeservicios,mpagomiscuentas ) , na.rm=TRUE ) ]

 
    #Comisiones
    dataset[ , ccomisiones                := rowSums( cbind( ccomisiones_mantenimiento,ccomisiones_otras ) , na.rm=TRUE ) ]
    dataset[ , mcomisiones                := rowSums( cbind( mcomisiones_mantenimiento,mcomisiones_otras ) , na.rm=TRUE ) ]


    #Cheques
    dataset[ , ccheques                := rowSums( cbind( ccheques_depositados,ccheques_emitidos ) , na.rm=TRUE ) ]
    dataset[ , mcheques                := rowSums( cbind( mcheques_depositados,mcheques_emitidos ) , na.rm=TRUE ) ]
  
    #Cajeros
    dataset[ , catm_total_trx                := rowSums( cbind(catm_trx,catm_trx_other ) , na.rm=TRUE ) ]
    dataset[ , matm_total_trx                := rowSums( cbind(matm,matm_other ) , na.rm=TRUE ) ]
    
    dataset[ , ccualquier_cajero_trx                := rowSums( cbind(catm_total_trx,cextraccion_autoservicio ) , na.rm=TRUE ) ]
    dataset[ , mcualquier_cajero_trx                := rowSums( cbind(matm_total_trx,mextraccion_autoservicio ) , na.rm=TRUE ) ]
    dataset[ , usa_cualquier_cajero             := ifelse(ccualquier_cajero_trx > 0,1,0)]

     #Payroll
   dataset[ , cpayroll_total    := rowSums( cbind(cpayroll_trx,cpayroll2_trx ) , na.rm=TRUE ) ]
   dataset[ , mpayroll_total    := rowSums( cbind(mpayroll,mpayroll2 ) , na.rm=TRUE ) ]
    dataset[ , acredita_sueldo  := ifelse(cpayroll_total > 0,1,0)]

  
  
    #Rentabilidad y productos
    dataset[ , mmargen    := rowSums( cbind(mactivos_margen,mpasivos_margen ) , na.rm=TRUE ) ]

   
    #Debito y credito
    dataset[ , ctarjeta_credito    := rowSums( cbind(ctarjeta_visa,ctarjeta_master ) , na.rm=TRUE ) ]
    dataset[ , ctarjeta_credito_transacciones    := rowSums( cbind(ctarjeta_visa_transacciones,ctarjeta_master_transacciones ) , na.rm=TRUE ) ]
    dataset[ , mtarjeta_credito_consumos    := rowSums( cbind(mtarjeta_visa_consumo,mtarjeta_master_consumo ) , na.rm=TRUE ) ]

    dataset[ , usa_tarjeta_credito  := ifelse(ctarjeta_credito_transacciones > 0,1,0)]
    dataset[ , usa_tarjeta_debito  := ifelse(ctarjeta_debito_transacciones > 0,1,0)]

    dataset[ , prop_tarj_debito_credito    := ifelse(ctarjeta_credito_transacciones==0,0,ctarjeta_debito_transacciones/ctarjeta_credito_transacciones) ]

    #Saldo
    dataset[ , prop_dolares_saldo   := ifelse(mcuentas_saldo==0,0,mcaja_ahorro_dolares/mcuentas_saldo) ]
    dataset[ , prop_plazos_fijos_saldo   := ifelse(mcuentas_saldo==0,0,mplazo_fijo/mcuentas_saldo) ]
    dataset[ , prop_inversion12_saldo   := ifelse(mcuentas_saldo==0,0,minversion12/mcuentas_saldo) ]
    dataset[ , prop_consumos_tarj_cred_saldo    := ifelse(mcuentas_saldo==0,0,mtarjeta_credito_consumos/mcuentas_saldo) ]
    dataset[ , prop_rent_mes_saldo   := ifelse(mcuentas_saldo==0,0,mrentabilidad/mcuentas_saldo) ]

    dataset[ , mcuentas_saldo_mas_invers    := rowSums( cbind(mcuentas_saldo,minversion_total ) , na.rm=TRUE ) ]
    dataset[ , mcuentas_saldo_mas_invers_mas_caja_seg    := ifelse(ccaja_seguridad==1,1000000,0) + mcuentas_saldo_mas_invers ]
    dataset[ , mgasto_mensual_estim    := rowSums( cbind(mdebitos_automaticos,mpago_servicios_total,mtransferencias_emitidas,mextraccion_autoservicio,mcheques_emitidos,ccajas_extracciones,mautoservicio,mtarjeta_credito_consumos ) , na.rm=TRUE ) ]
    dataset[ , mingreso_mensual_estim    := rowSums( cbind(mtransferencias_recibidas,mcheques_depositados,ccajas_depositos,mpayroll_total ) , na.rm=TRUE ) ]
    
    dataset[ , prop_gasto_saldo   := ifelse(mcuentas_saldo==0,0,mgasto_mensual_estim/mcuentas_saldo) ]
    dataset[ , prop_ingreso_saldo   := ifelse(mcuentas_saldo==0,0,mingreso_mensual_estim/mcuentas_saldo) ]

      #Mes
    dataset[ , mes   := str_sub(foto_mes, start=-2)  ]

    
############# FIN FE 1 ####################
    
    
    
    
    
    
    
    
    
    
    
    
    
    
  #valvula de seguridad para evitar valores infinitos
  #paso los infinitos a NULOS
  infinitos      <- lapply(names(dataset),function(.name) dataset[ , sum(is.infinite(get(.name)))])
  infinitos_qty  <- sum( unlist( infinitos) )
  if( infinitos_qty > 0 )
  {
    cat( "ATENCION, hay", infinitos_qty, "valores infinitos en tu dataset. Seran pasados a NA\n" )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  #valvula de seguridad para evitar valores NaN  que es 0/0
  #paso los NaN a 0 , decision polemica si las hay
  #se invita a asignar un valor razonable segun la semantica del campo creado
  nans      <- lapply(names(dataset),function(.name) dataset[ , sum(is.nan(get(.name)))])
  nans_qty  <- sum( unlist( nans) )
  if( nans_qty > 0 )
  {
    cat( "ATENCION, hay", nans_qty, "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n" )
    cat( "Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

}

In [10]:
#primero agrego las variables manuales
AgregarVariables( dataset )

ATENCION, hay 91 valores infinitos en tu dataset. Seran pasados a NA


In [11]:
fwrite( dataset,
        file=paste0("./datasets/",kdatasetoutput),
        sep= "," )

In [12]:
ncol(dataset)

[1] 240

In [13]:
nrow(dataset)

[1] 4214386